In [1]:
import os
import cv2
import itertools
import numpy as np
import mediapipe as mp

In [2]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

### 1. 솔루션 import

In [3]:
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
mp_face_mesh

<module 'mediapipe.python.solutions.face_mesh' from 'C:\\ProgramData\\Anaconda3\\envs\\GMIC_env_py36\\lib\\site-packages\\mediapipe\\python\\solutions\\face_mesh.py'>

In [51]:
# LEFT_EYE_INDEXES = list(set(itertools.chain(*mp_face_mesh.FACEMESH_LEFT_EYE)))
# LEFT_EYEBROW = list(set(itertools.chain(*mp_face_mesh.FACEMESH_LEFT_EYEBROW)))

# RIGHT_EYE_INDEXES = list(set(itertools.chain(*mp_face_mesh.FACEMESH_RIGHT_EYE)))

# LIPS_INDEXES = list(set(itertools.chain(*mp_face_mesh.FACEMESH_LIPS)))

#TESLE = list(set(itertools.chain(*mp_face_mesh.FACEMESH_TESSELATION)))

CONTOURS_INDEXES = list(set(itertools.chain(*mp_face_mesh.FACE_CONNECTIONS)))

#FACE_LINE  = list(set(itertools.chain(*mp_face_mesh.FACEMESH_FACE_OVAL)))

In [22]:
def to_pixel_coords(relative_coords):
    return(tuple(round(coord*dimension) for coord, dimension in zip(relative_coords, SCREEN_DIMENSIONS)))

In [23]:
def get_point(face_landmarks, location, idx, axis):
    coor = face_landmarks.landmark[location[idx]]
    pix = to_pixel_coords((coor.x, coor.y))
    if axis=='x':
        point = pix[0]
    elif axis=='y':
        point = pix[1]
    return point

#### *좌표 순서 표시

In [46]:
# 하나의 이미지에 대해 좌표 순서 표시
path = 'C:'
image = cv2.imread(path + 'blue.png', cv2.IMREAD_UNCHANGED)
image_input = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

height, width, _ = image.shape
SCREEN_DIMENSIONS = (width, height)
results = face_mesh.process(cv2.cvtColor(image_input, cv2.COLOR_BGR2RGB))

for face_landmarks in results.multi_face_landmarks: 
    for i, face_line_index in enumerate(CONTOURS_INDEXES):
        coordi = face_landmarks.landmark[face_line_index]
        coord = (coordi.x, coordi.y)
        #SCREEN_DIMENSIONS = (image_cols, image_rows)
        SCREEN_DIMENSIONS = (width, height)
        
        pixel = to_pixel_coords(coord)
        cv2.putText(image_input, str(i), pixel, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
    cv2.imwrite(path + 'blue_test.png', image_input)

### Face detection & Cropping

In [28]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

path = 'C:'
images = os.listdir(path)

for image_name in images:
    image = cv2.imread(path + image_name, cv2.IMREAD_UNCHANGED)
    
    height, width, _ = image.shape          
    SCREEN_DIMENSIONS = (width, height)

    results = face_mesh.process(cv2.cvtColor(image , cv2.COLOR_BGR2RGB))
                                                                                                                          #image = cv2.imread(path + image_name)

    for face_landmarks in results.multi_face_landmarks:
        
        startX = int(get_point(face_landmarks, CONTOURS_INDEXES, 65, axis='x')-(0.05*width))
        startY = int(get_point(face_landmarks, CONTOURS_INDEXES, 2, axis='y')-(0.09*height))
        endX = int(get_point(face_landmarks, CONTOURS_INDEXES, 126, axis='x')+(0.05*width))
        endY = int(get_point(face_landmarks, CONTOURS_INDEXES, 47, axis='y')+(0.05*height))
        
        if endY > height:
            endY = height
            
        if (startY < 0):
            startY = 0
            
        if endX > width:
            endX = width
            
        if (startX < 0):
            startX = 0
        
        cv2.rectangle(image, (startX, startY), (endX, endY), (0, 0, 0), 3)
        face_img = image[startY:endY, startX:endX].copy()
        
        #cv2.imwrite('C:/Users/dbssk6904/PycharmProjects/DEMODEX/typical_TL/1. typical/preprocessed_data/1. face_detect/' + image_name, image)
        cv2.imwrite('C:' + image_name, face_img)

### 2. 비식별화

In [1]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

path = 'C:'
crop_images = os.listdir(path)

lip_order = [0,70,71,72,79,105,92,123,89,5,27,62,30,43,17,63,10,9,8]

for image_name in crop_images:
    image = cv2.imread(path + image_name, cv2.IMREAD_UNCHANGED)   # image 열고 저장할 때, 자꾸 돌아가는 사진 있는데 이거 방지하기 위해 cv2.IMREAD_UNCHANGED
    
    height, width, _ = image.shape          
    SCREEN_DIMENSIONS = (width, height)

    results = face_mesh.process(cv2.cvtColor(image , cv2.COLOR_BGR2RGB))
    image = cv2.imread(path + image_name, cv2.IMREAD_UNCHANGED)
    print(image_name)
    
    lip_coordinates = []
    for face_landmarks in results.multi_face_landmarks:
        
        # right eye
        left = get_point(face_landmarks, CONTOURS_INDEXES, 7, axis='x')
        right = get_point(face_landmarks, CONTOURS_INDEXES, 39, axis='x')
        top = get_point(face_landmarks, CONTOURS_INDEXES, 53, axis='y')
        bottom = get_point(face_landmarks, CONTOURS_INDEXES, 42, axis='y')

        center_x = left + int((right-left)/2)
        center_y = top + int((bottom-top)/2)

        img = cv2.ellipse(image, (int(center_x*0.95), center_y), 
                          (int((right-center_x)*1.3), int((bottom-center_y)*1.8)), 0, 0, 360, (0,0,0), -1)

        # left eye
        left = get_point(face_landmarks, CONTOURS_INDEXES, 101, axis='x')
        right = get_point(face_landmarks, CONTOURS_INDEXES, 69, axis='x')
        top = get_point(face_landmarks, CONTOURS_INDEXES, 114, axis='y')
        bottom = get_point(face_landmarks, CONTOURS_INDEXES, 104, axis='y')

        center_x = left + int((right-left)/2)
        center_y = top + int((bottom-top)/2)  

        img = cv2.ellipse(img, (int(center_x*1.02), center_y), 
                          (int((right-center_x)*1.3), int((bottom-center_y)*1.8)), 0, 0, 360, (0,0,0), -1)

        # lip
        for idx in lip_order:
            coordi = face_landmarks.landmark[CONTOURS_INDEXES[idx]]
            coord = [coordi.x, coordi.y]

            pix = list(to_pixel_coords((coordi.x, coordi.y)))

            lip_coordinates.append(pix)

    pts = np.array(lip_coordinates, np.int32)

    img = cv2.fillPoly(image, [pts], (0,0,0), cv2.LINE_AA)
        
    cv2.imwrite('C:' + image_name, img)

### 3. 전체 패턴 이미지

참고 : https://stackoverflow.com/questions/48301186/cropping-concave-polygon-from-image-using-opencv-python

In [2]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

path = 'C:'
images = os.listdir(path)

idx_order = [2, 36, 21, 33, 14, 6, 56, 37, 65, 31, 38, 16, 58, 40, 
         46, 45, 60, 44, 47, 106, 121, 107, 108, 102, 119, 78, 100, 93, 126, 
         99, 117, 68, 76, 95, 83, 98]

for image_name in images:
    image = cv2.imread(path + image_name, cv2.IMREAD_UNCHANGED)
    
    height, width, _ = image.shape          
    SCREEN_DIMENSIONS = (width, height)

    results = face_mesh.process(cv2.cvtColor(image , cv2.COLOR_BGR2RGB))
    image = cv2.imread(path + image_name, cv2.IMREAD_UNCHANGED)
    print(image_name)
    
    coordinates = []
    for face_landmarks in results.multi_face_landmarks:
        for idx in idx_order:
            coordi = face_landmarks.landmark[CONTOURS_INDEXES[idx]]
            coord = [coordi.x, coordi.y]

            pix = list(to_pixel_coords((coordi.x, coordi.y)))

            coordinates.append(pix)
    
    pts = np.array(coordinates, np.int32)
    mask = np.zeros(image.shape[:2], np.uint8)
    
    cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)
    
    img = cv2.bitwise_or(image, image, mask=mask)

    cv2.imwrite('C:' + image_name, img)
    

### 4. Patches

In [48]:
base_path = 'C:'

In [52]:
# 코 부분 패치

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

path = base_path + '2. crop/'
crop_images = os.listdir(path)

for image_name in crop_images:
    image = cv2.imread(path + image_name, cv2.IMREAD_UNCHANGED)
    
    height, width, _ = image.shape          
    SCREEN_DIMENSIONS = (width, height)

    results = face_mesh.process(cv2.cvtColor(image , cv2.COLOR_BGR2RGB))
    image = cv2.imread(path + image_name, cv2.IMREAD_UNCHANGED)

    for face_landmarks in results.multi_face_landmarks:
        w1 = get_point(face_landmarks, CONTOURS_INDEXES, 39, axis='x')
        w2 = get_point(face_landmarks, CONTOURS_INDEXES, 101, axis='x')
        h1 = get_point(face_landmarks, CONTOURS_INDEXES, 35, axis='y')
        h1 = int(h1*0.95)
        h2 = get_point(face_landmarks, CONTOURS_INDEXES, 100, axis='y') 
        
        if h1<0:
            h1=0

        #print(h1, h2, w1, w2)
        h = h1 + int((h2-h1) / 2)

        patch_1 = image[h1:h, w1:w2]
        patch_2 = image[h:h2, w1:w2]

        cv2.imwrite(base_path + 'patch1/' + image_name, patch_1)
        cv2.imwrite(base_path + 'patch2/' + image_name, patch_2)

In [53]:
# 오른쪽 볼 패치

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

path = base_path + '2. crop/'
crop_images = os.listdir(path)

for image_name in crop_images:
    image = cv2.imread(path + image_name, cv2.IMREAD_UNCHANGED)
    
    height, width, _ = image.shape          
    SCREEN_DIMENSIONS = (width, height)

    results = face_mesh.process(cv2.cvtColor(image , cv2.COLOR_BGR2RGB))
    image = cv2.imread(path + image_name, cv2.IMREAD_UNCHANGED)

    for face_landmarks in results.multi_face_landmarks:
        w1 = get_point(face_landmarks, CONTOURS_INDEXES, 7, axis='x')
        w1 = int(w1*0.9)
        
        w2 = get_point(face_landmarks, CONTOURS_INDEXES, 39, axis='x')
        
        h1 = get_point(face_landmarks, CONTOURS_INDEXES, 42, axis='y')
        h1 = int(h1*1.02)
        
        h2 = get_point(face_landmarks, CONTOURS_INDEXES, 9, axis='y')
        h2 = int(h2*0.98)

        #print(h1, h2, w1, w2)
        h = h1 + int((h2-h1)/2)

        patch_3 = image[h1:h, w1:w2]
        patch_4 = image[h:h2, w1:w2]
        
        cv2.imwrite(base_path + 'patch3/' + image_name, patch_3)
        cv2.imwrite(base_path + 'patch4/' + image_name, patch_4)

In [54]:
# 왼쪽 볼 패치

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

path = base_path + '2. crop/'
crop_images = os.listdir(path)

for image_name in crop_images:
    image = cv2.imread(path + image_name, cv2.IMREAD_UNCHANGED)
    
    height, width, _ = image.shape          
    SCREEN_DIMENSIONS = (width, height)

    results = face_mesh.process(cv2.cvtColor(image , cv2.COLOR_BGR2RGB))
    image = cv2.imread(path + image_name, cv2.IMREAD_UNCHANGED)

    for face_landmarks in results.multi_face_landmarks:
        w1 = get_point(face_landmarks, CONTOURS_INDEXES, 101, axis='x')
        
        w2 = get_point(face_landmarks, CONTOURS_INDEXES, 69, axis='x')
        w2 = int(w2*1.05)
        
        h1 = get_point(face_landmarks, CONTOURS_INDEXES, 104, axis='y')
        h1 = int(h1*1.02)
        
        h2 = get_point(face_landmarks, CONTOURS_INDEXES, 71, axis='y')
        h2 = int(h2*0.98)

        #print(h1, h2, w1, w2)
        h = h1 + int((h2-h1)/2)

        patch_5 = image[h1:h, w1:w2]
        patch_6 = image[h:h2, w1:w2]
        
        cv2.imwrite(base_path + 'patch5/' + image_name, patch_5)
        cv2.imwrite(base_path + 'patch6/' + image_name, patch_6)

In [57]:
# 턱 패치

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

path = base_path + '2. crop/'
images = os.listdir(path)

idx_order = [40, 46, 45, 60, 44, 47, 106, 121, 107, 108, 102]
lip_idx_order = [79, 124, 72, 71, 70, 0, 8, 9, 10, 63]

for image_name in images:
    image = cv2.imread(path + image_name, cv2.IMREAD_UNCHANGED)
    
    height, width, _ = image.shape          
    SCREEN_DIMENSIONS = (width, height)

    results = face_mesh.process(cv2.cvtColor(image , cv2.COLOR_BGR2RGB))
    image = cv2.imread(path + image_name, cv2.IMREAD_UNCHANGED)
    
    coordinates = []
    for face_landmarks in results.multi_face_landmarks:
        for idx in idx_order:
            coordi = face_landmarks.landmark[CONTOURS_INDEXES[idx]]
            coord = [coordi.x, coordi.y]
            pix = list(to_pixel_coords((coordi.x, coordi.y)))
            coordinates.append(pix)
            
        for lip_idx in lip_idx_order:
            lip_coordi = face_landmarks.landmark[CONTOURS_INDEXES[lip_idx]]
            lip_coord = [lip_coordi.x, lip_coordi.y]
            lip_pix = list(to_pixel_coords((lip_coordi.x, (lip_coordi.y)-0.03)))
            coordinates.append(lip_pix)
    
    pts = np.array(coordinates, np.int32)
    mask = np.zeros(image.shape[:2], np.uint8)
    
    cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)
    
    img = cv2.bitwise_and(image, image, mask=mask)
    patch_7 = img[coordinates[15][1]-10:height, coordinates[0][0]-10:coordinates[10][0]+10]

    cv2.imwrite(base_path + 'patch7/' + image_name, patch_7)
    